In [50]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [ ]:
import math

radius = 1
height = 0.5
velocity = 10  # tangential velocity
omega = velocity / radius  # angular velocity

# distribut 3 points on a circle
xs = [radius * math.cos(2 * math.pi / 3 * i) for i in range(3)]
ys = [radius * math.sin(2 * math.pi / 3 * i) for i in range(3)]

def circular_velocity(x, y, omega):
    # Perpendicular to (x, y) in the x-y plane, magnitude = omega * radius
    return (-y * omega, x * omega, 0)

m1Vel = circular_velocity(xs[0], ys[0], omega)
m2Vel = circular_velocity(xs[1], ys[1], omega)
m3Vel = circular_velocity(xs[2], ys[2], omega)

# to compute distance between two 3D points
def distance(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2)

In [52]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)


m1Pos = (xs[0], ys[0], height)
m1Vel = circular_velocity(xs[0], ys[0], omega)
m2Pos = (xs[1], ys[1], height)
m2Vel = circular_velocity(xs[1], ys[1], omega)
m3Pos = (xs[2], ys[2], height)
m3Vel = circular_velocity(xs[2], ys[2], omega)

# mBottom = mss.add (Mass(1, (0,0,0)))
m1 = mss.add (Mass(1, m1Pos))
m2 = mss.add (Mass(1, m2Pos))
m3 = mss.add (Mass(1, m3Pos))

f1 = mss.add (Fix( (0,0,0)) )

mss.add (Spring(distance((0,0,0), m1Pos), 100000, (f1, m1)))
mss.add (Spring(distance((0,0,0), m2Pos), 100000, (f1, m2)))
mss.add (Spring(distance((0,0,0), m3Pos), 100000, (f1, m3)))


mss.add(Spring(distance(m1Pos, m2Pos), 100000, (m1, m2)))
mss.add(Spring(distance(m2Pos, m3Pos), 100000, (m2, m3)))
mss.add(Spring(distance(m3Pos, m1Pos), 100000, (m3, m1)))

#  void setState (VectorView<> values, VectorView<> dvalues, VectorView<> ddvalues)

mss.setState(
    [m1Pos, m2Pos, m3Pos],
    [m1Vel, m2Vel, m3Vel],
    [[0,0,0], [0,0,0], [0,0,0]]
)

In [53]:
print("Velocity vectors:   ")
print("m1Vel:", m1Vel)
print("m2Vel:", m2Vel)
print("m3Vel:", m3Vel)

Velocity vectors:   
m1Vel: (-0.0, 1.0, 0)
m2Vel: (-0.8660254037844387, -0.4999999999999998, 0)
m3Vel: (0.8660254037844384, -0.5000000000000004, 0)


In [54]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [55]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [56]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

KeyboardInterrupt: 